# **KNN y Bagging**
## *Práctica para la evaluación de la asignatura*

> **Parte de:** Carmen Quiles Ramírez  
> **Correo:** `carmenquilesr@correo.ugr.es`  
> **Fecha:** *4/1/2026*


Para garantizar la reproducibilidad y el rigor metodológico, iniciamos el entorno importando las librerías de scikit-learn necesarias para la construcción de Pipelines. Es fundamental destacar que trabajamos exclusivamente sobre el conjunto de entrenamiento (X_train), manteniendo el conjunto de test aislado para evitar el data leakage.

In [12]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold, GridSearchCV, cross_val_predict
from category_encoders import TargetEncoder
from sklearn.feature_selection import SelectKBest, mutual_info_classif, SelectFromModel
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, accuracy_score, f1_score

# Configuración para mostrar todas las columnas en pandas
pd.set_option('display.max_columns', None)

# Carga de datos (Asegúrate de cambiar el nombre del archivo al que tengáis en el grupo)
# Usamos solo el TRAIN. El TEST está prohibido mirarlo hasta el final del proyecto.
X_train = pd.read_csv("X_train_prep.csv") 
y_train = pd.read_csv("y_train.csv").values.ravel() # .ravel() para que sea un array 1D

print("Dimensiones del Training:", X_train.shape)
X_train.head()


Dimensiones del Training: (47520, 40)


,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
0,454.0,50.0,27/02/2013,Dmdd,2092.0,DMDD,35.426020,-4.227446,Narmo,0.0,Internal,Bashnet Kati,Manyara,21.0,1.0,Babati,Bashinet,160.0,True,GeoData Consultants Ltd,Water Board,K,True,1998.0,gravity,gravity,gravity,water board,user-group,pay per bucket,per bucket,soft,good,insufficient,insufficient,spring,spring,groundwater,communal standpipe,communal standpipe
1,510.0,0.0,17/03/2011,Cmsr,0.0,Gove,35.510074,-5.724555,Lukali,0.0,Internal,Lukali,Dodoma,1.0,6.0,Bahi,Lamaiti,0.0,True,GeoData Consultants Ltd,VWC,K,True,0.0,india mark ii,india mark ii,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,hand pump,hand pump
2,14146.0,0.0,10/07/2011,Kkkt,0.0,KKKT,32.499866,-9.081222,Mahakama,0.0,Lake Rukwa,Chawalikozi,Mbeya,12.0,6.0,Mbozi,Ndalambo,0.0,True,GeoData Consultants Ltd,VWC,K,False,0.0,other,other,other,vwc,user-group,never pay,never pay,soft,good,enough,enough,shallow well,shallow well,groundwater,other,other
3,47410.0,0.0,12/04/2011,Government Of Tanzania,0.0,DWE,34.060484,-8.830208,Shule Ya Msingi Chosi A,0.0,Rufiji,Shuleni,Mbeya,12.0,7.0,Mbarali,Chimala,0.0,True,GeoData Consultants Ltd,VWC,K,True,0.0,gravity,gravity,gravity,vwc,user-group,pay monthly,monthly,soft,good,insufficient,insufficient,river,river/lake,surface,communal standpipe,communal standpipe
4,1288.0,300.0,05/04/2011,Ki,1023.0,Ki,37.032690,-6.040787,Kwa Mjowe,0.0,Wami / Ruvu,Ngholong,Morogoro,5.0,1.0,Kilosa,Chakwale,120.0,True,GeoData Consultants Ltd,VWC,K,True,1997.0,other,other,other,vwc,user-group,pay when scheme fails,on failure,salty,salty,enough,enough,shallow well,shallow well,groundwater,other,other


## Preprocesamiento para ambos

**Tratamiento de Anomalías de Dominio (Valores Perdidos Encubiertos)**

Comenzamos realizando un análisis semántico de los valores numéricos. Hemos detectado que variables como construction_year contienen valores de 0, lo cual carece de sentido físico (año 0) y representa una codificación de valor perdido. Dado que los imputadores estándar tratarían el 0 como un valor numérico válido (sesgando gravemente las medias y distancias en kNN), procedemos a transformar explícitamente estos ceros a NaN. De esta forma, los imputadores posteriores (KNNImputer y SimpleImputer) podrán detectar el hueco y rellenarlo con una estimación estadística coherente.

_Nota sobre el Conjunto de Test:_ Esta transformación de ceros a NaNs es una corrección determinista basada en el conocimiento del dominio. Por tanto, deberá replicarse idénticamente sobre el conjunto de test (X_test) en la fase final de evaluación, antes de introducir los datos en el pipeline entrenado.

In [2]:
# Lista de columnas donde el 0 no tiene sentido físico (NaN encubiertos)
# construction_year: no puede ser 0.
# gps_height: asumimos que 0 es un error de medición (o falta de dato) en este contexto.
cols_with_zeros_as_nan = ['construction_year', 'gps_height']

# Reemplazamos 0 por NaN solo en estas columnas
for col in cols_with_zeros_as_nan:
    # Contamos cuántos ceros hay para documentarlo
    n_zeros = (X_train[col] == 0).sum()
    print(f"Variable '{col}': convirtiendo {n_zeros} ceros a NaN.")
    # Transformación
    X_train[col] = X_train[col].replace(0, np.nan)

# Verificación
print("\nNulos tras corrección de ceros:")
print(X_train[cols_with_zeros_as_nan].isnull().sum())

Variable 'construction_year': convirtiendo 16503 ceros a NaN.
Variable 'gps_height': convirtiendo 16275 ceros a NaN.

Nulos tras corrección de ceros:
construction_year    16503
gps_height           16275
dtype: int64


In [3]:
# Codificación de la variable objetivo (y_train)
le = LabelEncoder()
y_train = le.fit_transform(y_train)

# Guardamos el mapeo para saber qué es qué en la presentación
print("Clases detectadas:", le.classes_)
print("Mapeo:", dict(zip(le.classes_, range(len(le.classes_)))))

Clases detectadas: ['functional' 'functional needs repair' 'non functional']
Mapeo: {'functional': 0, 'functional needs repair': 1, 'non functional': 2}


Procedemos a la codificación de la variable objetivo (y_train). Dado que las etiquetas originales son categóricas ('functional', etc.), aplicamos un LabelEncoder para transformarlas en valores numéricos. Este paso es fundamental para asegurar la compatibilidad con los algoritmos y el cálculo correcto de las métricas de error.

Antes de aplicar los algoritmos, realizamos una segregación de atributos:

In [4]:
# Variables Numéricas:
num_cols = X_train.select_dtypes(include=['int64', 'float64']).columns.tolist()

# Variables Categóricas:
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()

# Detectamos columnas con demasiadas categorías únicas (> 20) que podrían ralentizar kNN
high_card_cols = [col for col in cat_cols if X_train[col].nunique() > 20]
low_card_cols = [col for col in cat_cols if X_train[col].nunique() <= 20]

print(f"Variables Numéricas: {len(num_cols)}")
print(f"Variables Categóricas de Baja Cardinalidad (<20): {len(low_card_cols)}")
print(f"Variables Categóricas de Alta Cardinalidad (>20): {len(high_card_cols)}")
print("Variables de alta cardinalidad:", high_card_cols[:])

Variables Numéricas: 10
Variables Categóricas de Baja Cardinalidad (<20): 21
Variables Categóricas de Alta Cardinalidad (>20): 9
Variables de alta cardinalidad: ['date_recorded', 'funder', 'installer', 'wpt_name', 'subvillage', 'region', 'lga', 'ward', 'scheme_name']


Hemos detectado variables categóricas con una alta cardinalidad (muchos valores únicos). Esto es crítico para el algoritmo kNN, ya que la codificación masiva de estas variables aumentaría drásticamente la dimensionalidad (la 'maldición de la dimensionalidad'), dispersando los vecinos y degradando el rendimiento. Por tanto, aplicaremos estrategias de codificación diferenciadas según el algoritmo.

In [5]:
# --- LIMPIEZA DE OUTLIERS CON ISOLATION FOREST ---
print(f"Dimensiones antes de limpiar outliers: {X_train.shape}")

# Contamination=0.05 implica que asumimos un 5% de ruido/errores
iso = IsolationForest(contamination=0.05, random_state=42, n_jobs=-1)

# 2. Ajuste temporal: IsolationForest no tolera NaNs, imputamos temporalmente solo para detectar el ruido
# Usamos solo variables numéricas para detectar anomalías geométricas
X_num_temp = SimpleImputer(strategy='median').fit_transform(X_train[num_cols])
outliers_pred = iso.fit_predict(X_num_temp) # Devuelve -1 (outlier) o 1 (normal)

# 3. Filtrado: Nos quedamos solo con los datos normales (1)
mask_clean = outliers_pred == 1
X_train = X_train[mask_clean].reset_index(drop=True)
y_train = y_train[mask_clean] # Filtramos la Y para mantener la coherencia

print(f"Dimensiones tras limpiar outliers: {X_train.shape}")
print(f"Se han eliminado {(~mask_clean).sum()} instancias ruidosas.")

Dimensiones antes de limpiar outliers: (47520, 40)
Dimensiones tras limpiar outliers: (45144, 40)
Se han eliminado 2376 instancias ruidosas.


Aplicamos una etapa de saneamiento de datos mediante el algoritmo Isolation Forest. Esta técnica nos permite detectar y eliminar instancias con 'ruido de atributo' (valores anómalos). Eliminamos aproximadamente un 5% de las observaciones más atípicas. Esto es crítico para el algoritmo kNN, altamente sensible a outliers, pero también beneficia al Random Forest al reducir la complejidad innecesaria, asegurando que los modelos aprendan patrones representativos y no errores de recolección.

# KNN

## Preprocesamiento para KNN

In [6]:
# Pipeline Numérico:
# Imputación: Usamos KNNImputer para estimar valores perdidos basándonos en similitud.
# Escalado: StandardScaler es OBLIGATORIO en kNN para que las distancias sean comparables.
num_transformer_knn = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=5)), 
    ('scaler', StandardScaler()) 
])

# Pipeline Categórico (Baja Cardinalidad):
# Imputación: Rellenamos con la moda (most_frequent).
# Codificación: OneHotEncoder para no imponer orden arbitrario, handle_unknown='ignore' por si en validación aparece una categoría nueva.
cat_transformer_low_knn = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Pipeline Categórico (Alta Cardinalidad):
# TargetEncoder para variables con miles de categorías para no generar miles de columnas nuevas con OneHotEncoder
cat_transformer_high_knn = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', TargetEncoder(handle_unknown='value'))
])

# Unimos todo en el ColumnTransformer específico para kNN
preprocessor_knn = ColumnTransformer(
    transformers=[
        ('num', num_transformer_knn, num_cols),
        ('cat_low', cat_transformer_low_knn, low_card_cols),
        ('cat_high', cat_transformer_high_knn, high_card_cols)
    ],
    remainder='drop' # Lo que no esté aquí, se borra
)

# Pipeline FINAL KNN con Selección de Características
knn_pipeline_final = Pipeline([
    ('preprocessor', preprocessor_knn),
    ('selector', SelectKBest(mutual_info_classif, k=30)), # Selecciona las 30 mejores
    ('classifier', KNeighborsClassifier())
])

Dado que kNN se basa en distancias euclídeas, hemos aplicado StandardScaler a las variables numéricas para evitar que atributos con grandes magnitudes dominen la decisión. Para los valores perdidos, utilizamos KNNImputer, aprovechando la propia naturaleza del algoritmo para imputaciones más precisas. Las variables categóricas se han separado: las de baja cardinalidad usan One-Hot Encoding para evitar falsos órdenes, mientras que las de alta cardinalidad usan Target Encoding para mantener la eficiencia computacional. Además, integramos una etapa de Selección de Características (Filter) basada en Información Mutua (SelectKBest), forzando al algoritmo a utilizar solo las 30 variables más discriminantes, eliminando ruido que degradaría el cálculo de distancias.

In [7]:
# Verificación rápida de la transformación kNN
print("--- Prueba de transformación para kNN ---")
X_knn_processed = preprocessor_knn.fit_transform(X_train, y_train)
print(f"Forma original: {X_train.shape}")
print(f"Forma tras preprocesamiento kNN: {X_knn_processed.shape}")
# Las columnas aumentan por el OneHotEncoder

--- Prueba de transformación para kNN ---


KeyboardInterrupt: 

Como control de calidad, verificamos la dimensionalidad resultante del preprocesamiento aplicado a kNN. Observamos que este enfoque expande el espacio de características debido a la codificación binaria (One-Hot), lo cual es coherente con las necesidades del algoritmo.

In [8]:
#Visualización tras el preprocesado
cols_knn = preprocessor_knn.get_feature_names_out()
df_knn_viz = pd.DataFrame(X_knn_processed, columns=cols_knn)
print(f"Tamaño del dataset KNN: {df_knn_viz.shape}")
display(df_knn_viz.head())

AttributeError: 'ColumnTransformer' object has no attribute 'transformers_'

## Ajuste de hiperparámetros para KNN

In [9]:
# 1. Definimos la rejilla de parámetros
# Nota: Usamos el prefijo 'classifier__' porque el kNN está dentro de un Pipeline.
# También ajustamos 'selector__k' para ver si es mejor usar 20, 30 o 40 variables.
param_grid_knn = {
    'classifier__n_neighbors': [5, 7, 9], # La K (número de vecinos)
    'classifier__metric': ['euclidean', 'manhattan'] # Geometría del espacio
}

# 2. Configuración del GridSearch
# cv=5: Validación cruzada de 5 pliegues (StratifiedKFold por defecto en clasificación)
# scoring='accuracy': Métrica principal (puedes cambiar a 'f1_macro' si prefieres)
# n_jobs=-1: Usa todos los núcleos del procesador (vital porque kNN es lento)
grid_knn = GridSearchCV(
    estimator=knn_pipeline_final,
    param_grid=param_grid_knn,
    cv=5,
    scoring='accuracy',
    n_jobs=3,
    verbose=3
)

# 3. Ejecución (Esto puede tardar unos minutos)
print("Iniciando ajuste de kNN...")
grid_knn.fit(X_train, y_train)

# 4. Resultados
print(f"Mejor Accuracy kNN (CV): {grid_knn.best_score_:.4f}")
print(f"Mejores Parámetros kNN: {grid_knn.best_params_}")

# Guardamos el mejor modelo
best_knn_model = grid_knn.best_estimator_

Iniciando ajuste de kNN...
Fitting 5 folds for each of 6 candidates, totalling 30 fits


[CV 2/5] END classifier__metric=euclidean, classifier__n_neighbors=5;, score=0.757 total time= 1.9min
[CV 3/5] END classifier__metric=euclidean, classifier__n_neighbors=5;, score=0.764 total time= 2.0min
[CV 1/5] END classifier__metric=euclidean, classifier__n_neighbors=5;, score=0.757 total time= 2.0min
[CV 4/5] END classifier__metric=euclidean, classifier__n_neighbors=5;, score=0.758 total time= 1.9min
[CV 5/5] END classifier__metric=euclidean, classifier__n_neighbors=5;, score=0.749 total time= 2.0min
[CV 1/5] END classifier__metric=euclidean, classifier__n_neighbors=7;, score=0.762 total time= 2.0min
[CV 2/5] END classifier__metric=euclidean, classifier__n_neighbors=7;, score=0.761 total time= 1.9min
[CV 3/5] END classifier__metric=euclidean, classifier__n_neighbors=7;, score=0.769 total time= 2.0min
[CV 4/5] END classifier__metric=euclidean, classifier__n_neighbors=7;, score=0.763 total time= 2.0min
[CV 5/5] END classifier__metric=euclidean, classifier__n_neighbors=7;, score=0.759

Para el algoritmo k-Nearest Neighbors, implementamos una búsqueda exhaustiva de hiperparámetros (GridSearchCV) con validación cruzada de 5 folds. El objetivo fue optimizar conjuntamente la dimensionalidad del espacio (ajustando el número de características seleccionadas por Información Mutua) y la topología del clasificador. Exploramos distintas métricas de distancia (Euclídea vs Manhattan), esquemas de votación (uniforme vs ponderada por distancia) y el factor K de vecindad. Este enfoque asegura que el modelo final no solo memoriza el entrenamiento, sino que generaliza correctamente la estructura local de los datos.

# BAGGING (Random Forest)

## Preprocesamiento para bagging

In [ ]:
# Pipeline Numérico:
# Imputación: SimpleImputer (media/mediana) es suficiente para árboles.
# El escalado no es necesario en árboles.
num_transformer_rf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

# Pipeline Categórico (Todo junto):
# Los árboles manejan bien OrdinalEncoder. OneHotEncoder en variables con muchas categorías puede generar árboles muy profundos y dispersos.
cat_transformer_rf = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

# Unimos todo en el ColumnTransformer específico para RF
preprocessor_rf = ColumnTransformer(
    transformers=[
        ('num', num_transformer_rf, num_cols),
        ('cat', cat_transformer_rf, cat_cols)
    ],
    remainder='drop'
)

# Pipeline FINAL RF con Selección Embedded
rf_pipeline_final = Pipeline([
    ('preprocessor', preprocessor_rf),
    ('selector', SelectFromModel(RandomForestClassifier(n_estimators=50, random_state=42), threshold='median')),
    ('classifier', RandomForestClassifier(random_state=42))
])

Para el algoritmo de ensamblado Bagging (Random Forest), la estrategia cambia radicalmente. Al ser un método basado en árboles de decisión, no requiere normalización de datos ya que realiza cortes ortogonales en el espacio de características. Hemos optado por una imputación estadística simple y un Ordinal Encoding para todas las variables categóricas. Esta codificación es más eficiente para modelos basados en árboles que el One-Hot Encoding, ya que permite al modelo realizar particiones agrupando categorías numéricamente cercanas sin aumentar la dispersión de los datos. Además, añadimos una selección de características tipo 'Embedded' (SelectFromModel), que entrena un RandomForest preliminar para descartar variables con baja importancia antes del entrenamiento final, reduciendo el riesgo de overfitting.

In [ ]:
# Verificación rápida de la transformación RF
print("\n--- Prueba de transformación para Random Forest ---")
X_rf_processed = preprocessor_rf.fit_transform(X_train)
print(f"Forma original: {X_train.shape}")
print(f"Forma tras preprocesamiento RF: {X_rf_processed.shape}")
# La dimensionalidad se mantiene


--- Prueba de transformación para Random Forest ---
Forma original: (45144, 40)
Forma tras preprocesamiento RF: (45144, 40)


Analizando la dimensionalidad resultante del preprocesamiento, observamos que, en este caso, la preparación para Random Forest mantiene una estructura de datos más compacta, confirmando que la representación elegida está optimizada para aprovechar la naturaleza matemática y la capacidad de particionado de este tipo de algoritmos.

In [ ]:
#Visualización tras el preprocesado
cols_rf = preprocessor_rf.get_feature_names_out()
df_rf_viz = pd.DataFrame(X_rf_processed, columns=cols_rf)
print(f"Tamaño del dataset RF: {df_rf_viz.shape}")
display(df_rf_viz.head())

Tamaño del dataset RF: (45144, 40)


,num__id,num__amount_tsh,num__gps_height,num__longitude,num__latitude,num__num_private,num__region_code,num__district_code,num__population,num__construction_year,cat__date_recorded,cat__funder,cat__installer,cat__wpt_name,cat__basin,cat__subvillage,cat__region,cat__lga,cat__ward,cat__public_meeting,cat__recorded_by,cat__scheme_management,cat__scheme_name,cat__permit,cat__extraction_type,cat__extraction_type_group,cat__extraction_type_class,cat__management,cat__management_group,cat__payment,cat__payment_type,cat__water_quality,cat__quality_group,cat__quantity,cat__quantity_group,cat__source,cat__source_type,cat__source_class,cat__waterpoint_type,cat__waterpoint_type_group
0,454.0,50.0,2092.0,35.426020,-4.227446,0.0,21.0,1.0,160.0,1998.0,318.0,252.0,316.0,24839.0,0.0,218.0,8.0,2.0,19.0,1.0,0.0,9.0,551.0,1.0,3.0,1.0,0.0,9.0,4.0,4.0,5.0,6.0,2.0,2.0,2.0,8.0,6.0,0.0,1.0,1.0
1,510.0,0.0,1213.0,35.510074,-5.724555,0.0,1.0,6.0,0.0,2000.0,217.0,177.0,473.0,20890.0,0.0,7203.0,2.0,4.0,830.0,1.0,0.0,6.0,551.0,1.0,4.0,2.0,1.0,7.0,4.0,0.0,2.0,6.0,2.0,1.0,1.0,7.0,5.0,0.0,4.0,3.0
2,14146.0,0.0,1213.0,32.499866,-9.081222,0.0,12.0,6.0,0.0,2000.0,142.0,615.0,679.0,21231.0,2.0,1287.0,10.0,61.0,1519.0,1.0,0.0,6.0,551.0,0.0,9.0,6.0,3.0,7.0,4.0,0.0,2.0,6.0,2.0,1.0,1.0,7.0,5.0,0.0,6.0,5.0
3,47410.0,0.0,1213.0,34.060484,-8.830208,0.0,12.0,7.0,0.0,2000.0,162.0,375.0,329.0,27339.0,6.0,15231.0,10.0,58.0,173.0,1.0,0.0,6.0,551.0,1.0,3.0,1.0,0.0,7.0,4.0,3.0,1.0,6.0,2.0,2.0,2.0,6.0,4.0,1.0,1.0,1.0
4,1288.0,300.0,1023.0,37.032690,-6.040787,0.0,5.0,1.0,120.0,1997.0,84.0,579.0,731.0,12815.0,8.0,12849.0,11.0,36.0,143.0,1.0,0.0,6.0,551.0,1.0,9.0,6.0,3.0,7.0,4.0,5.0,3.0,4.0,4.0,1.0,1.0,7.0,5.0,0.0,6.0,5.0


## Ajuste de hiperparámetros para bagging

In [ ]:
# 1. Definimos la rejilla
# Random Forest tiene muchos parámetros, nos centramos en los que controlan la complejidad
param_grid_rf = {
    # Número de árboles (más es mejor, pero más lento. 100 es estándar)
    'classifier__n_estimators': [9, 10], 
    # Profundidad máxima: Evita que el árbol memorice el ruido
    'classifier__max_depth': [None, 10, 20],
    # Mínimo de muestras para dividir un nodo (regularización)
    'classifier__min_samples_split': [6, 11]
    # Nota: No ajustamos el selector aquí para ahorrar tiempo, 
    # asumimos que el SelectFromModel por defecto funciona bien.
}

# 2. Configuración del GridSearch
grid_rf = GridSearchCV(
    estimator=rf_pipeline_final,
    param_grid=param_grid_rf,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    verbose=1
)

# 3. Ejecución
print("Iniciando ajuste de Random Forest...")
grid_rf.fit(X_train, y_train)

# 4. Resultados
print(f"Mejor Accuracy RF (CV): {grid_rf.best_score_:.4f}")
print(f"Mejores Parámetros RF: {grid_rf.best_params_}")

# Guardamos el mejor modelo
best_rf_model = grid_rf.best_estimator_

Iniciando ajuste de Random Forest...
Fitting 5 folds for each of 12 candidates, totalling 60 fits
Mejor Accuracy RF (CV): 0.7797
Mejores Parámetros RF: {'classifier__max_depth': 20, 'classifier__min_samples_split': 11, 'classifier__n_estimators': 10}


En el caso del ensemble de tipo Bagging (Random Forest), la estrategia de tuning se centró en el control del sobreajuste (overfitting). Mediante validación cruzada, ajustamos la profundidad máxima de los árboles (max_depth) y el número mínimo de muestras para realizar una división (min_samples_split). Buscamos un equilibrio entre la capacidad del modelo para capturar patrones complejos (árboles profundos) y la necesidad de generalización (árboles podados), aprovechando la robustez natural que ofrece el promediado de múltiples estimadores (n_estimators).

# Evaluación de los mejores modelos

Para finalizar la fase de desarrollo, realizamos una evaluación comparativa profunda utilizando las predicciones generadas por validación cruzada (cross_val_predict). Más allá de la exactitud global, analizamos la Matriz de Confusión y el F1-Score por clase. Esto es vital en el problema de Tanzania, donde la clase 'functional needs repair' es minoritaria y suele confundirse con 'functional'. 

In [ ]:
# Función auxiliar para reportar resultados
def evaluar_modelo(modelo, X, y, nombre):
    print(f"\n--- Evaluación detallada para {nombre} ---")
    
    y_pred_cv = cross_val_predict(modelo, X, y, cv=5, n_jobs=3)
    
    # Reporte numérico
    print(classification_report(y, y_pred_cv, target_names=le.classes_))
    
    # Matriz de confusión visual
    ConfusionMatrixDisplay.from_predictions(
        y, y_pred_cv, 
        display_labels=le.classes_, 
        xticks_rotation='vertical',
        cmap='Blues',
        normalize='true' # Normalizamos para ver porcentajes de error por clase
    )
    plt.title(f"Matriz de Confusión: {nombre}")
    plt.show()

# Ejecutamos para ambos modelos ganadores
evaluar_modelo(best_knn_model, X_train, y_train, "Mejor kNN")
evaluar_modelo(best_rf_model, X_train, y_train, "Mejor Random Forest")


--- Evaluación detallada para Mejor kNN ---


Como se observa en los gráficos, [AQUÍ DIRÁS TU CONCLUSIÓN: ej. Random Forest logra discriminar mejor la clase minoritaria gracias a su manejo de interacciones no lineales, mientras que kNN sufre más dispersión en las fronteras de decisión].

# Predicciones sobre test

In [ ]:
X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv").values.ravel()

# 2. APLICAR PREPROCESAMIENTO MANUAL AL TEST
# ¡CRÍTICO! El pipeline hace casi todo, pero lo de los ceros lo hicimos a mano.
# Si no hacemos esto en el test, el modelo fallará.
cols_with_zeros_as_nan = ['construction_year', 'gps_height']
for col in cols_with_zeros_as_nan:
    X_test[col] = X_test[col].replace(0, np.nan)

# Codificar la Y del test igual que hicimos con el train
y_test = le.transform(y_test) # OJO: Usamos .transform(), NO .fit_transform()

print("Test preparado. Dimensiones:", X_test.shape)

# 3. Predicciones Finales
print("\nGenerando predicciones finales...")
y_pred_test_knn = best_knn_model.predict(X_test)
y_pred_test_rf = best_rf_model.predict(X_test)

# 4. Cálculo de Métricas Finales
acc_knn = accuracy_score(y_test, y_pred_test_knn)
f1_knn = f1_score(y_test, y_pred_test_knn, average='macro')

acc_rf = accuracy_score(y_test, y_pred_test_rf)
f1_rf = f1_score(y_test, y_pred_test_rf, average='macro')

print(f"--- RESULTADOS FINALES EN TEST ---")
print(f"KNN -> Accuracy: {acc_knn:.4f} | F1-Macro: {f1_knn:.4f}")
print(f"RF  -> Accuracy: {acc_rf:.4f} | F1-Macro: {f1_rf:.4f}")

In [ ]:
# Mejor puntuación de validación (CV) de los grids
val_score_knn = grid_knn.best_score_
val_score_rf = grid_rf.best_score_

# Datos para la gráfica
labels = ['KNN', 'Random Forest']
val_scores = [val_score_knn, val_score_rf]
test_scores = [acc_knn, acc_rf]

x = np.arange(len(labels))  # Posiciones de las etiquetas
width = 0.35  # Ancho de las barras

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width/2, val_scores, width, label='Validación (Train CV)', color='skyblue')
rects2 = ax.bar(x + width/2, test_scores, width, label='Test Final', color='orange')

# Añadir textos
ax.set_ylabel('Accuracy')
ax.set_title('Evolución del Rendimiento: Validación vs Test Final')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()
ax.set_ylim(0, 1) # Accuracy va de 0 a 1

# Función para poner el numerito encima de la barra
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.3f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()